One of the applications of convex optimization in complex variables is phase recovery, a problem in signal processing. The phase recovery problem seeks to retrieve a signal $x \in \mathbb{C}^p$ from the amplitude b = |Ax| of n linear measurements. The mathematical reprsentation of the problem is as follows:

>>>> find x

>>>> such that |Ax| = b

>>>> where $x \in \mathbb{C}^p$, $A \in \mathbb{C}^{n×p}$ and $b \in \mathbb{R}^n$.

This problem is orginally a nonconvex optimization problem. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

Given a linear operator $A$ and a vector $b= |Ax|$ of measured amplitudes, define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.

In [95]:
using Convex

n = 8
p = 2
A = rand(n,p) + im*randn(n,p)
b = rand(n)
M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((16,16),(16,16))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 505
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 129, constraints m = 273
Cones:	primal zero / dual free vars: 137
	sd vars: 136, sd blks: 1
Setup time: 4.07e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  1.94e-04 
   100| 2.43e-03  1.11e-01  1.61e-05 -4.42e+01 -4.42e+01  1.70e-16  2.03e-02 
   200| 1.98e-03  1.12e-01  1.40e-04 -4.44e+01 -4.44e+01  1.33e-16  4.69e-02 


8×8 Array{Complex{Float64},2}:
 1.0-1.93739e-17im           0.950097-0.311954im   …     0.994131-0.108179im 
     0.950097+0.311954im  1.0+2.2594e-17im               0.978268+0.207343im 
    -0.988573-0.150744im    -0.986265+0.165168im        -0.999079-0.0429163im
     0.970355+0.241684im     0.997326-0.0730829im        0.990806+0.135294im 
    -0.113401-0.993549im    -0.417683-0.908593im        -0.220216-0.975451im 
    -0.967851-0.251525im    -0.998017+0.0629521im  …    -0.989381-0.145348im 
    -0.210569-0.977579im    -0.505021-0.863107im        -0.315087-0.949063im 
     0.994131+0.108179im     0.978268-0.207343im      1.0-1.1868e-16im       

In [96]:
# Verify if the rank of U is 1
B, C = eig(U.value);
length([e for e in B if(abs(real(e))>1e-4)])
#Decompose U = uu*
u = C[:,1]
for i in 1:n
        u[i] = u[i]/abs(u[i])
end
atan(sum(imag(u))/sum(real(u)))

-1.0526255468882988

In [97]:
x = A'*diagm(b)*u
atan(sum(imag(x))/sum(real(x)))

1.5027502381165467